AWS-Account: 678856817733
Region: US-West-2

https://us-west-2.console.aws.amazon.com/sagemaker/home?region=us-west-2#/
https://us-west-2.console.aws.amazon.com/sagemaker/home?region=us-west-2#/studio/open/d-warwybct01ai/default-20230911t173777

https://github.com/aws-samples/amazon-bedrock-workshop




In [ ]:
# Make sure you ran `download-dependencies.sh` from the root of the repository first!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet "faiss-cpu>=1.7,<2" langchain==0.0.249 "pypdf>=3.8,<4"
%pip install unstructured


In [ ]:
nstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet "faiss-cpu>=1.7,<2" langchain==0.0.249 "pypdf>=3.8,<4"
%pip install unstructured


In [52]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


os.environ["AWS_DEFAULT_REGION"] = "us-west-2"
os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:iam::678856817733:role/BedrockRole"
os.environ["BEDROCK_ENDPOINT_URL"] = "https://bedrock.us-west-2.amazonaws.com"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-west-2
  Using role: arn:aws:iam::678856817733:role/BedrockRole ... successful!
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-west-2.amazonaws.com)


In [80]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm = Bedrock(model_id="anthropic.claude-v1", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':1000})
bedrock_embeddings = BedrockEmbeddings(client=boto3_bedrock)

## Data Preparation

In [81]:
from urllib.request import urlretrieve

os.makedirs("data", exist_ok=True)
files = [
    "https://raw.githubusercontent.com/wiki/nuvibit/semper-policy-repo-sample/Home.md",
    "https://raw.githubusercontent.com/wiki/nuvibit/semper-policy-repo-sample/10-SEMPER-Policies.md",
    "https://raw.githubusercontent.com/wiki/nuvibit/semper-policy-repo-sample/90-JSON-Engine.md",
]
for url in files:
    file_path = os.path.join("data", url.rpartition("/")[2])
    urlretrieve(url, file_path)

In [87]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.document_loaders import DirectoryLoader

def load_md_from_directory(directory_path):
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.md'):  # Assuming you want to read .txt files
            loader = UnstructuredMarkdownLoader(os.path.join(directory_path, filename))
            documents.append(loader.load())
    return documents

loader = DirectoryLoader("./data/", glob="**/*.md")

documents = loader.load()
# - in our testing Character split works better with this PDF data set
text_splitter = CharacterTextSplitter(
    # Set a really small chunk size, just to show.
    separator = "\n",
    chunk_size = 750,
    chunk_overlap  = 0,
)
docs = text_splitter.split_documents(documents)

In [88]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

Average length among 3 documents loaded is 14180 characters.
After the split we have 61 documents more than the original 3.
Average length among 61 documents (after split) is 687 characters.


In [89]:
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectorstore_faiss = FAISS.from_documents(
    docs,
    bedrock_embeddings,
)

wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

In [90]:
query = "I want to get a enrichment policy to trigger an auto-remediation for the open security group port TCP80."

In [91]:
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

array([ 0.05932617,  0.04003906,  0.296875  , ..., -0.15625   ,
        0.02954102,  0.01116943])

In [92]:
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')
for i, rel_doc in enumerate(relevant_documents):
    print_ww(f'## Document {i+1}: {rel_doc.page_content}.......')
    print('---')

4 documents are fetched which are relevant to the query.
----
## Document 1: | ...findingResource    | object | (optional) |
| ....id                | string | (optional, default = '') you can self-reference the JSON of the
processed Security Finding.
Example: 'raw.detail.requestParameters.keyId' |
| ....type              | string | (optional, default = 'AwsAccount') according to
AWS documentation. |
Samples of Extension-Policies  🔝
Sample 1 🔝
json {linenos=table,hl_lines=[],linenostart=50}
{
  "metaData": {
    "domain": "extension",
    "title": "Auto-Remediation of TCP-Ports 22 & 3389 for CIDR range /24",
    ...
  },
  "extension": {
    "policyScope": {
      ...
    },
    "findingPattern": {
      ...
    },
    "extensionBlock": {
      "sqsFanOut": [
        {.......
---
## Document 2: "findingComplianceStatus" : "FAILED",
      }
    }
  }
}
Processed Security Finding  🔝
A processed Security Finding will sent to one of two SNS topics.
The JSON of the processed Security Findin

In [93]:
answer = wrapper_store_faiss.query(question=query, llm=llm)
print_ww(answer)

ValueError: Error raised by bedrock service: An error occurred (AccessDeniedException) when calling the InvokeModel operation: Your account is not authorized to invoke this API operation.